# Kernel Semántico con Integración del Servidor OpenBnB MCP

Este cuaderno muestra cómo usar Kernel Semántico con el servidor OpenBnB MCP real para buscar alojamientos reales de Airbnb utilizando MCPStdioPlugin. Para el acceso a LLM, utiliza Azure AI Foundry. Para configurar tus variables de entorno, puedes seguir la [Lección de Configuración](/00-course-setup/README.md)


## Importar los Paquetes Necesarios


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Creando la Conexión del Plugin MCP

Nos conectaremos al [servidor MCP de OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) utilizando MCPStdioPlugin. Este servidor ofrece funcionalidad de búsqueda de Airbnb a través del paquete @openbnb/mcp-server-airbnb.


## Creando el Cliente

En este ejemplo, utilizaremos Azure AI Foundry para acceder a nuestro LLM. Asegúrate de que tus variables de entorno estén configuradas correctamente.


## Configuración del Entorno

Configura los ajustes de Azure OpenAI. Asegúrate de tener las siguientes variables de entorno configuradas:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Entendiendo la Integración de OpenBnB MCP

Este cuaderno se conecta al **servidor real de OpenBnB MCP** que proporciona funcionalidad real de búsqueda en Airbnb.

### Cómo funciona:

1. **MCPStdioPlugin**: Utiliza comunicación de entrada/salida estándar con el servidor MCP
2. **Paquete NPM Real**: Descarga y ejecuta `@openbnb/mcp-server-airbnb` mediante npx
3. **Datos en Vivo**: Devuelve datos reales de propiedades de Airbnb desde sus APIs
4. **Descubrimiento de Funciones**: El agente descubre automáticamente las funciones disponibles del servidor MCP

### Funciones Disponibles:

El servidor MCP de OpenBnB típicamente proporciona:
- **search_listings** - Buscar propiedades de Airbnb por ubicación y criterios
- **get_listing_details** - Obtener información detallada sobre propiedades específicas
- **check_availability** - Verificar disponibilidad para fechas específicas
- **get_reviews** - Recuperar reseñas de propiedades
- **get_host_info** - Obtener información sobre los anfitriones de las propiedades

### Requisitos Previos:

- **Node.js** instalado en tu sistema
- **Conexión a Internet** para descargar el paquete del servidor MCP
- **NPX** disponible (incluido con Node.js)

### Probando la Conexión:

Puedes probar el servidor MCP manualmente ejecutando:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Esto descargará y arrancará el servidor MCP de OpenBnB, al cual Semantic Kernel se conecta para obtener datos reales de Airbnb.


## Ejecutar el Agente con el Servidor OpenBnB MCP

Ahora ejecutaremos el Agente de IA que se conecta al servidor OpenBnB MCP para buscar alojamientos reales de Airbnb en Estocolmo para 2 adultos y 1 niño. Siéntete libre de cambiar la lista `user_inputs` para modificar los criterios de búsqueda.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Resumen
¡Felicidades! Has construido con éxito un agente de IA que se integra con la búsqueda de alojamientos en el mundo real utilizando el Protocolo de Contexto de Modelo (MCP):

## Tecnologías Utilizadas:
- Semantic Kernel - Para crear agentes inteligentes con Azure OpenAI
- Azure AI Foundry - Para capacidades de modelos de lenguaje y finalización de chat
- MCP (Protocolo de Contexto de Modelo) - Para integración estandarizada de herramientas
- OpenBnB MCP Server - Para funcionalidad real de búsqueda en Airbnb
- Node.js/NPX - Para ejecutar el servidor MCP externo

## Lo que Has Aprendido:
- Integración MCP: Conectar agentes de Semantic Kernel a servidores MCP externos
- Acceso a Datos en Tiempo Real: Buscar propiedades reales de Airbnb a través de APIs en vivo
- Comunicación por Protocolo: Usar comunicación stdio entre el agente y el servidor MCP
- Descubrimiento de Funciones: Descubrir automáticamente funciones disponibles de servidores MCP
- Respuestas en Streaming: Capturar y registrar llamadas a funciones en tiempo real
- Renderizado HTML: Formatear respuestas del agente con tablas estilizadas y pantallas interactivas

## Próximos Pasos:
- Integrar servidores MCP adicionales (clima, vuelos, restaurantes)
- Construir un sistema multi-agente combinando protocolos MCP y A2A
- Crear servidores MCP personalizados para tus propias fuentes de datos
- Implementar memoria de conversación persistente entre sesiones
- Desplegar el agente en Azure Functions con orquestación de servidores MCP
- Agregar autenticación de usuarios y capacidades de reserva



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
